In [4]:
import ee
import requests

ee.Initialize()

def getSTAC(ic: ee.ImageCollection) -> dict:
    """Gets the STAC of the image.""" 

    # extract the Asset id from the image collection
    assetId = ic.get("system:id").getInfo()

    # search for the project in the GEE catalog and extract the project catalog URL
    project = assetId.split("/")[0]
    catalog = "https://earthengine-stac.storage.googleapis.com/catalog/catalog.json"
    links = requests.get(catalog).json()["links"]
    project_catalog = next((i["href"] for i in links if i.get("title") == project), None)
    if project_catalog is None:
        raise ValueError(f"Project {project} not found in the catalog")

    # search for the collection in the project catalog and extract the collection STAC URL
    collection = "_".join(assetId.split("/"))
    links = requests.get(project_catalog).json()["links"]
    collection_stac = next((i["href"] for i in links if i.get("title") == collection), None)
    if collection_stac is None:
        raise ValueError(f"Collection {collection} not found in the {project} catalog")

    return requests.get(collection_stac).json()

def get_last_date(collection): 
    ic = ee.ImageCollection(collection)
    stac = getSTAC(ic)
    return stac["extent"]["temporal"]["interval"][0][1]

list_collection = [
    "COPERNICUS/S2_HARMONIZED",
]

date_dict = {c: get_last_date(c) for c in list_collection}
print(date_dict)


{'COPERNICUS/S2_HARMONIZED': '2024-06-02T18:11:12Z'}
